In [ ]:
import marimo as mo

# 極大ナップサック問題

## 問題

正整数の配列 $a = [ a_1, \dots, a_n]$ と整数 $\tau$ が与えられる.
このとき $a$ の部分配列でその**総積**が $\tau$ 以下になるもので極大なものを全て列挙せよ.

## 定式化

$x = [x_1, \dots, x_n]$ を 0-1 決定変数とする.
$x_i$ が $1$ のとき $a_i$ を採用し, $0$ のとき採用しない.
また, 決定変数 $y = [y_1, \dots, y_n]$ を用意し $y_i = (a_i - 1) x_i + 1$ を課す.
こうすると $y_i$ は $x_i$ が $1$ のとき $a_i$ を値に取り, $0$ のとき $1$ を値にとる.
同様に $z_i = (a_i - 1) (1 - x_i) + 1$ とおく.
こちらは $y_i$ とは逆に採用されてない $a_i$ に対してだけ値 $a_i$ を取り, 採用されているものについては $1$ を取る.

Google OR-Tools は整数変数の積をそのまま扱えるため, 線形ではない定式化を行う.

- $\prod_{i=1}^n y_i \leq \tau$: 採用した $a_i$ の積が $\tau$ を超えない.
- $z_j \prod_{i=1}^n y_i > \tau (1 - x_j) \quad (\forall j = 1, \dots, n)$: 採用しなかった $a_j$ を掛けたら $\tau$ を超えてしまう.

また, Google OR-Tools の CP-SAT ソルバーは目的関数を設定しない場合,
実行可能解を全て求めるよう指示できるのでそれを使って全ての極大元を求める.

## 実装

In [ ]:
from ortools.sat.python import cp_model
import math
import time

In [ ]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, a, solution: list[cp_model.IntVar]):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution = solution
        self.__a = a
        self.__solution_count = 0
        self.__all_solutions = []
        self.__start_time = time.time()

    @property
    def solution_count(self) -> int:
        return self.__solution_count

    @property
    def all_solutions(self):
        return self.__all_solutions

    def on_solution_callback(self):
        current_time = time.time()
        self.__solution_count = self.__solution_count + 1
        print(f'Solution {self.__solution_count}, time = {current_time - self.__start_time} s')
        print('  [', end=' ')
        for x in self.__solution:
            print(f'{self.value(x)}', end=' ')
        print(']')
        print('  [', end=' ')
        solution = []
        for i, x in enumerate(self.__solution):
            if self.value(x) == 1:
                solution.append(self.__a[i])
                print(f'{self.__a[i]}', end=' ')
        print(']')
        self.__all_solutions.append(solution)

In [ ]:
class Model:
    def __init__(self, a, tau):
        self.a = a
        self.tau = tau

        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()

        self.x = [self.model.new_bool_var(f"a{i} is used") for i in range(len(self.a))]
        y = [self.model.new_int_var(1, self.a[i], "") for i in range(len(self.a))]
        z = [self.model.new_int_var(1, self.a[i], "") for i in range(len(self.a))]

        for i in range(len(self.a)):
            self.model.add(y[i] == (self.a[i] - 1) * self.x[i] + 1)
            self.model.add(z[i] == (self.a[i] - 1) * (1 - self.x[i]) + 1)

        y_prod = self.model.new_int_var(1, tau, "")
        self.model.add_multiplication_equality(y_prod, y)
        for j in range(len(self.a)):
            y_prod_zj = self.model.new_int_var(1, tau * self.a[j], "")
            self.model.add_multiplication_equality(y_prod_zj, [z[j], y_prod])
            self.model.add(y_prod_zj >= self.tau * (1 - self.x[j]) + 1)

    def solve(self):
        self.solution_printer = SolutionPrinter(self.a, self.x)
        self.solver.parameters.enumerate_all_solutions = True
        #self.solver.parameters.log_search_progress = True
        self.solver.solve(self.model, self.solution_printer)

In [ ]:
# a = list(range(1, 5+1))
# tau = 10

# a = list(range(1, 10+1))
# tau = 50

a = list(range(1, 20+1))
tau = 100

# a = [2] * 20 # 終わらない
# tau = 2 ** 10

print(f"a = {a}")
print(f"tau = {tau}")

a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
tau = 100


In [ ]:
model = Model(a, tau)
model.solve()

Solution 1, time = 0.019679784774780273 s
  [ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 ]
  [ 1 2 20 ]
Solution 2, time = 0.029220104217529297 s
  [ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 ]
  [ 1 2 19 ]


Solution 3, time = 0.03765606880187988 s
  [ 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 6 7 ]
Solution 4, time = 0.043617963790893555 s
  [ 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 6 ]
Solution 5, time = 0.046922922134399414 s
  [ 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 5 ]


Solution 6, time = 0.051133155822753906 s
  [ 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ]
  [ 1 4 13 ]
Solution 7, time = 0.05532407760620117 s
  [ 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 8 ]
Solution 8, time = 0.057424068450927734 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 ]
  [ 1 5 14 ]
Solution 9, time = 0.05888700485229492 s
  [ 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 ]
  [ 1 4 14 ]
Solution 10, time = 0.06005597114562988 s
  [ 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 ]
  [ 1 6 14 ]


Solution 11, time = 0.06553483009338379 s
  [ 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 4 ]
Solution 12, time = 0.07052302360534668 s
  [ 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 6 9 ]
Solution 13, time = 0.07253694534301758 s
  [ 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 5 9 ]
Solution 14, time = 0.07389688491821289 s
  [ 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 4 9 ]
Solution 15, time = 0.07505393028259277 s
  [ 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 9 ]


Solution 16, time = 0.07918691635131836 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 ]
  [ 1 5 15 ]
Solution 17, time = 0.08150696754455566 s
  [ 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 ]
  [ 1 5 12 ]
Solution 18, time = 0.08482694625854492 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ]
  [ 1 5 13 ]
Solution 19, time = 0.08705401420593262 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 ]
  [ 1 5 20 ]


...

Solution 70, time = 0.41297197341918945 s
  [ 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 ]
  [ 1 7 13 ]
Solution 71, time = 0.42084503173828125 s
  [ 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 7 ]


Solution 72, time = 0.42584896087646484 s
  [ 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 ]
  [ 1 7 12 ]
Solution 73, time = 0.43099308013916016 s
  [ 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 5 7 ]


Solution 74, time = 0.4394559860229492 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 ]
  [ 1 5 18 ]


Solution 75, time = 0.4528648853302002 s
  [ 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 ]
  [ 1 8 11 ]
Solution 76, time = 0.459061861038208 s
  [ 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 ]
  [ 1 5 11 ]
Solution 77, time = 0.46144890785217285 s
  [ 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 ]
  [ 1 3 18 ]


Solution 78, time = 0.46491098403930664 s
  [ 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 ]
  [ 1 9 11 ]
Solution 79, time = 0.46686887741088867 s
  [ 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 ]
  [ 1 7 11 ]
Solution 80, time = 0.46729207038879395 s
  [ 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 ]
  [ 1 6 11 ]


In [ ]:
model.solution_printer.all_solutions

In [ ]:
print("Statistics")
print(f"  conflicts      : {model.solver.num_conflicts}")
print(f"  branches       : {model.solver.num_branches}")
print(f"  wall time      : {model.solver.wall_time} s")
print(f"  solutions found: {model.solution_printer.solution_count}")

Statistics
  conflicts      : 130
  branches       : 12601
  wall time      : 0.462936 s
  solutions found: 80


## 結果

掛け算を使った定式化は厳密ではあるものの数値が大きくなりすぎてしまう問題がある.
おとなしく $\log$ を使って線形計画問題にしよう.

## 定式化(MILP)

$x = [x_1, \dots, x_n]$ を 0-1 決定変数とする.
$x_i$ が $1$ のとき $a_i$ を採用し, $0$ のとき採用しない.

$$
w := \log(a_1) x_1 + \dots + \log(a_n) x_n
$$

とすると制約条件は以下のように書ける.

- $w \leq \log(\tau)$: 採用した $a_i$ の積が $\tau$ を超えない
- $w + \log(a_i) (1 - x_i) > \log(\tau) (1 - x_i) \quad (\forall i = 1, \dots, n)$: 採用しなかった $a_j$ を掛けたら $\tau$ を超えてしまう.

CP-SAT ソルバーは整数しか扱えないので $\log(a_i)$ や $\log(\tau)$ は適当にスケールして小数点以下を切り捨てることにする.
これによって誤差が出るかもしれない.

## 実装(MILP)

In [ ]:
class ModelLinear:
    def __init__(self, a, tau):
        self.a = a
        self.tau = tau
        base = 1000000
        self.log_a = [math.floor(math.log(ai) * base) for ai in a]
        self.log_tau = math.floor(math.log(self.tau) * base)
        print(f"log_a = {self.log_a}")
        print(f"log_tau = {self.log_tau}")
        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()
        self.x = [self.model.new_bool_var(f"a{i} is used") for i in range(len(self.a))]
        w = sum(a * x for a, x in zip(self.log_a, self.x))

        self.model.add(w <= self.log_tau)
        for a, x in zip(self.log_a, self.x):
            self.model.add(w + a * (1 - x) >= (self.log_tau + 1) * (1 - x))

    def solve(self):
        self.solution_printer = SolutionPrinter(self.a, self.x)
        self.solver.parameters.enumerate_all_solutions = True
        #self.solver.parameters.log_search_progress = True
        self.solver.solve(self.model, self.solution_printer)

In [ ]:
print(f"a = {a}")
print(f"tau = {tau}")

a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
tau = 100


In [ ]:
model_1 = ModelLinear(a, tau)
model_1.solve()

log_a = [0, 693147, 1098612, 1386294, 1609437, 1791759, 1945910, 2079441, 2197224, 2302585, 2397895, 2484906, 2564949, 2639057, 2708050, 2772588, 2833213, 2890371, 2944438, 2995732]
log_tau = 4605170
Solution 1, time = 0.0018808841705322266 s
  [ 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 5 ]
Solution 2, time = 0.001984119415283203 s
  [ 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 6 ]
Solution 3, time = 0.0020720958709716797 s
  [ 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 5 6 ]
Solution 4, time = 0.0021598339080810547 s
  [ 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 7 ]
Solution 5, time = 0.0022690296173095703 s
  [ 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 8 ]
Solution 6, time = 0.002359151840209961 s
  [ 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 7 8 ]
Solution 7, time = 0.0024509429931640625 s
  [ 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 6 9 ]
Solution 8, time = 0.0025348663330078125 s
  [ 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 

Solution 9, time = 0.0026891231536865234 s
  [ 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 8 9 ]
Solution 10, time = 0.002782106399536133 s
  [ 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 6 10 ]
Solution 11, time = 0.002864837646484375 s
  [ 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 7 10 ]
Solution 12, time = 0.0029468536376953125 s
  [ 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 8 10 ]
Solution 13, time = 0.0030279159545898438 s
  [ 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 9 10 ]
Solution 14, time = 0.003113985061645508 s
  [ 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 ]
  [ 1 5 11 ]
Solution 15, time = 0.0031960010528564453 s
  [ 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 ]
  [ 1 6 11 ]
Solution 16, time = 0.0032830238342285156 s
  [ 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 ]
  [ 1 7 11 ]
Solution 17, time = 0.0033648014068603516 s
  [ 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 ]
  [ 1 8 11 ]
Solution 18, time = 0.003448963165283203 s
  [ 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0

In [ ]:
model_1.solution_printer.all_solutions

In [ ]:
print('Statistics')
print(f' conflicts : {model_1.solver.num_conflicts}')
print(f' branches : {model_1.solver.num_branches}')
print(f' wall time : {model_1.solver.wall_time} s')
print(f' solutions found: {model_1.solution_printer.solution_count}')

Statistics
 conflicts : 14
 branches : 1186
 wall time : 0.009435 s
 solutions found: 80
